In [ ]:
quiet_library <- function(...) {
    suppressPackageStartupMessages(library(...))
}
quiet_library(Seurat)
quiet_library(ggplot2)
quiet_library(Matrix)
quiet_library(H5weaver)
quiet_library(dplyr)
quiet_library(viridis)
quiet_library(harmony)
quiet_library(Nebulosa)

In [ ]:
create_so <- function(path){
    mtx <- Read10X(data.dir = path)
    so <- CreateSeuratObject(mtx$`Gene Expression`, project = path)
    so[['ADT']] <- CreateAssayObject(mtx$`Antibody Capture`)
    return(so)
    }

In [ ]:
path_list <- c('published_datasets/cord_blood_young_adult/GSM4750303_F012/', 
               'published_datasets/cord_blood_young_adult/GSM4750304_F013/',
               'published_datasets/cord_blood_young_adult/GSM4750305_F014/',
               'published_datasets/cord_blood_young_adult/GSM4750306_F016/',
               'published_datasets/cord_blood_young_adult/GSM4750307_F017/',
               'published_datasets/cord_blood_young_adult/GSM4750308_F024/')

In [ ]:
so_list <- lapply(path_list, create_so)

In [ ]:
so_list

In [ ]:
so_merge <- Reduce(merge, so_list)
so_merge

In [ ]:
so_merge <- SCTransform(so_merge) %>% RunPCA()

In [ ]:
ElbowPlot(so_merge, ndims = 50)

In [ ]:
so_merge <- RunUMAP(so_merge, dims = 1:20) %>% FindNeighbors(dims = 1:20) %>% FindClusters(resolution = 0.5)

In [ ]:
DimPlot(so_merge)
DimPlot(so_merge, group.by = 'orig.ident')

In [ ]:
so_merge <- NormalizeData(so_merge, assay = 'ADT', normalization.method = 'CLR', margin = 2)

In [ ]:
rownames(so_merge[['ADT']])

In [ ]:
FeaturePlot(so_merge, features = 'adt_CD8-TotalSeqC', max.cutoff = 'q95')
FeaturePlot(so_merge, features = 'adt_CCR7-TotalSeqC', max.cutoff = 'q95')
FeaturePlot(so_merge, features = 'adt_CD4-TotalSeqC', max.cutoff = 'q95')
FeaturePlot(so_merge, features = 'sct_CD3G', max.cutoff = 'q95')

In [ ]:
ref <- readRDS(file = '/home/jupyter/reference/Hao-2021_PBMC-Multimodal-Reference_SeuratObject.rds')

In [ ]:
DimPlot(ref, group.by = 'celltype.l2')

In [ ]:
label_transfer_func_alt <- function(reference, query){
  anchors <- FindTransferAnchors(
    reference = reference,
    query = query,
    normalization.method = "SCT",
    reference.reduction = "spca",
    dims = 1:50, recompute.residuals = FALSE
  )
  query <- TransferData(anchorset = anchors, reference = reference, query = query,
                    refdata = list(
                      celltype.l1 = "celltype.l1",
                      celltype.l2 = "celltype.l2",
                      celltype.l3 = "celltype.l3",
                      predicted_ADT = 'ADT'))
  query
}

In [ ]:
so_merge <- label_transfer_func_alt(ref, so_merge)

In [ ]:
options(repr.plot.width = 12, repr.plot.height = 12)
DimPlot(so_merge, group.by = 'predicted.celltype.l2')

In [ ]:
saveRDS(so_merge, file = 'cb_ya_so.rds')